In [13]:
# -*- coding:utf-8 -*-
import argparse
import numpy as np
import torch.cuda
from sklearn.model_selection import KFold
from sklearn.metrics import f1_score
from glob import glob
from model import *
from utils import *
import pandas as pd
import os
from torch.utils.data import DataLoader
import argparse
import torch, gc

gc.collect()
torch.cuda.empty_cache()
result_dir='./sample_submission.csv'
data_dir='./test'
def get_predict_npy(cfg):
    dataset = pd.read_csv(result_dir,dtype={'Predicted':str})

    print(cfg)

    test = dataset.copy()
    test.columns = ['Id','Target']
    test['Id'] = data_dir +"/"+ test['Id']
    test['suffix'] = '.png'
    gen1 = HumanDataset(test)
    dataloader1 = DataLoader(gen1, batch_size=cfg['bs'], shuffle=False) # pin_memory=True,


    model= cfg['model'](cfg)
    if torch.cuda.is_available():
        model=model.cuda()

    result = []
    fold=1
    model.load_state_dict(torch.load(f"./weights/{cfg['name']}_fold{fold}.pkl"))
    _, pred1 = predict(model, dataloader1)
    result.append(pred1)
    result = np.average(result,axis=0)
    np.save(f"{cfg['name']}submit",result)

def main(cfg):
    dataset = pd.read_csv( './sample_submission.csv', dtype={'Predicted': str})
    result = np.load(f"{cfg['name']}submit.npy")

    target = []
    for sample_pred in result:
        pred = []
        for i,score in enumerate(sample_pred):
            if score > cfg['thres'][i]:
                pred.append(str(i))
        if len(pred) == 0:
            pred.append(str(sample_pred.argmax()))
        target.append(' '.join(pred))

    dataset['Predicted'] = target
    dataset.to_csv('submit.csv',index=False)

# os.path.join(data_dir,'00008af0-bad0-11e8-b2b8-ac1f6b6435d0')
# dataset = pd.read_csv(result_dir+r'\sample_submission.csv',dtype={'Predicted':str})
# test = dataset.copy()
# test.columns = ['Id','Target']
# test['Id'] =data_dir +"\\"+ test['Id']


if __name__ == '__main__':
    parser = argparse.ArgumentParser(description='Process some integers.')
    # parser.add_argument('--fold', type=str, required=True)
    # parser.add_argument('--name', type=str, required=True)
    # parser.add_argument('--device', type=str, required=True)
    parser.add_argument('-f', type=str, default="读取额外的参数")
    args = parser.parse_args()
    cfg = {}
    # cfg['device'] = [i for i in range(len(args.device))]
    cfg['model'] = xception_model
    cfg['lr'] = 0.0001
    cfg['backbone_lr'] = cfg['lr']
    cfg['bs'] = 24
    cfg['channel'] = 3
    cfg['name'] = "xception"
   
    cfg['thres'] = [0.3]*28

    get_predict_npy(cfg)
    main(cfg)

{'model': <function xception_model at 0x7f19d89c91f0>, 'lr': 0.0001, 'backbone_lr': 0.0001, 'bs': 24, 'channel': 3, 'name': 'xception', 'thres': [0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3]}


In [15]:
!nvidia-smi

Wed Dec 21 10:40:13 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.60.11    Driver Version: 525.60.11    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  Off  | 00000000:C1:00.0 Off |                  Off |
|  0%   42C    P8    20W / 450W |   6374MiB / 24564MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------